# Question-Answer System

In [1]:
import pandas as pd
from langchain.agents import create_pandas_dataframe_agent
from langchain import OpenAI
from prompts import REIDIN_PREFIX

In [3]:
df = pd.read_csv('data/new_reidin_data.csv')
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

In [4]:
llm = OpenAI(temperature=0.25, model_name='text-davinci-003', verbose=True)
agent = create_pandas_dataframe_agent(llm=llm, df=df, prefix=REIDIN_PREFIX)

In [5]:
agent.run('How many units in Villanova are Ready?')

'There are 2 Ready units in Villanova.'

In [ ]:
agent.run()